## Testing the API found by packet sniffing

The ASUS router has no public API. I found out the required REST API calls using a packet sniffer on my phone while running the ASUS router app. 

To turn off the VPN client and effectively connect to DE use 
```
    '{ 
     "vpnc_proto": "disable",
     "vpnc_pptp_options_x": "",
     "vpn_clientx_eas": "",
     "vpn_client_unit": "",
     "vpnc_pppoe_username": "",
     "vpnc_pppoe_passwd": "",
     "vpnc_heartbeat_x": "", 
     "action_mode": "apply" 
     }' 
```

To turn on vpn client 4 which is pre-configured to connect to the UK use:

```
    '{ 
     "vpnc_proto": "openvpn",
     "vpnc_pptp_options_x": "auto",
     "vpn_clientx_eas": "4,",
     "vpn_client_unit": "4",
     "vpn_client4_username": "my_user","vpn_client4_password": "my_pass", 
     "action_mode": "apply" 
     }'
```

After issuing the command, you have to apply it with:

```
    '{
     "action_mode": "apply",
     "rc_service": "restart_vpncall;"
     }'
```

We can find the current status of the VPN by issueing the following command and checking the response

```
    "hook=nvram_get(vpn_clientx_eas);')"
```

If the response is not Null then the value returned is the currently connected VPN


In [1]:
# First we have to get a token from the router to continue issuing other commands.

import requests
import base64

asus_ip = "192.168.1.1"
account =  "user:pass"

string_bytes = account.encode('ascii')
base64_bytes = base64.b64encode(string_bytes)
login = base64_bytes.decode('ascii')

url = 'http://{}/login.cgi'.format(asus_ip)
payload = "login_authorization=" + login
headers = {
    'user-agent': "asusrouter-Android-DUTUtil-1.0.0.245"
}
r = requests.post(url=url, data=payload, headers=headers)
token = r.json()['asus_token']
print(token)

eJEbSPn8SOooXL0hpWaJs4Hat6DS7G8


In [2]:
# Now we can use that token to find out other stuff. For example, here we find the total uptime. 
payload = "hook=uptime();')"
headers = {
    'user-Agent': "asusrouter-Android-DUTUtil-1.0.0.245",
    'cookie': 'asus_token={}'.format(token),
}
try:
    r = requests.post(url='http://{}/appGet.cgi'.format(asus_ip), data=payload, headers=headers)
except:
    print('Failed')
print(r.text)

{
"uptime":Fri, 01 Dec 2023 23:22:00 +0100(561 secs since boot)
}



In [5]:
#Turn off front panel LED
payload = "{ 'led_val': '0', 'action_mode': 'apply', 'rc_service': 'start_ctrl_led' };')"
headers = {
    'user-Agent': "asusrouter-Android-DUTUtil-1.0.0.245",
    'cookie': 'asus_token={}'.format(token),
}
try:
    r = requests.post(url='http://{}/applyapp.cgi'.format(asus_ip), data=payload, headers=headers)
except:
    print('Failed')
print(r.text)

{ "modify": "1", "run_service": "start_ctrl_led" }



In [ ]:
#Turn on front panel LED
payload = "{ 'led_val': '1', 'action_mode': 'apply', 'rc_service': 'start_ctrl_led' };')"
headers = {
    'user-Agent': "asusrouter-Android-DUTUtil-1.0.0.245",
    'cookie': 'asus_token={}'.format(token),
}
try:
    r = requests.post(url='http://{}/applyapp.cgi'.format(asus_ip), data=payload, headers=headers)
except:
    print('Failed')
print(r.text)

In [78]:
#Turn off vpn (no password required)
payload = '{ "vpnc_proto": "disable", "vpnc_pptp_options_x": "", "vpn_clientx_eas": "", "vpn_client_unit": "", "vpnc_pppoe_username": "", "vpnc_pppoe_passwd": "", "vpnc_heartbeat_x": "", "action_mode": "apply" };")'
headers = {
    'user-Agent': "asusrouter-Android-DUTUtil-1.0.0.245",
    'cookie': 'asus_token={}'.format(token),
}
try:
    r = requests.post(url='http://{}/applyapp.cgi'.format(asus_ip), data=payload, headers=headers)
except:
    print('Failed')
print(r.text)



{ "modify": "1" }



In [76]:
#Turn on UK VPN (VPN service password required)

payload = '{ "vpnc_proto": "openvpn", "vpnc_pptp_options_x": "auto", "vpn_clientx_eas": "5,", "vpn_client_unit": "5", "vpn_client5_username": "vpn_user", "vpn_client5_password": "vpn_pass", "action_mode": "apply" };")'
headers = {
    'user-Agent': "asusrouter-Android-DUTUtil-1.0.0.245",
    'cookie': 'asus_token={}'.format(token),
}
try:
    r = requests.post(url='http://{}/applyapp.cgi'.format(asus_ip), data=payload, headers=headers)
except:
    print('Failed')
print(r.text)



{ "modify": "1" }



In [79]:
#restart vpn service (required to apply any changes to the router/vpn client)

payload = '{ "action_mode": "apply", "rc_service": "restart_vpncall;" }")'
headers = {
    'user-Agent': "asusrouter-Android-DUTUtil-1.0.0.245",
    'cookie': 'asus_token={}'.format(token),
}
try:
    r = requests.post(url='http://{}/applyapp.cgi'.format(asus_ip), data=payload, headers=headers)
except:
    print('Failed')
print(r.text)


{ "modify": "0", "run_service": "restart_vpncall;" }



In [72]:
# Request the vpn client state. The return value is the currently enable client. If there is no return value, the vpn is not connected. 

payload = "hook=nvram_get(vpn_clientx_eas);')"
headers = {
    'user-Agent': "asusrouter-Android-DUTUtil-1.0.0.245",
    'cookie': 'asus_token={}'.format(token),
}
try:
    r = requests.post(url='http://{}/appGet.cgi'.format(asus_ip), data=payload, headers=headers)
except:
    print('Failed')
response = r.json()

if "vpn_clientx_eas" in response:
    if response["vpn_clientx_eas"] =="5,":
        print("Connected")
    else:
        print("Not connected")

    

Not connected
